In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random 
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available", len(physical_devices))

Num GPUs Available 1


In [3]:
# path to dataset
directory = "./FACD_image"
target_size = (224,224)
seed = 42;
batch_size=10

# create a image generator for keras, that can load images batchwise
data_generator = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input, validation_split=0.2)

train_batches = tf.keras.preprocessing.image.DirectoryIterator(
    directory, data_generator, target_size=target_size, color_mode='rgb', batch_size=batch_size, shuffle=True, seed=seed,
    subset='training'
)

valid_batches = tf.keras.preprocessing.image.DirectoryIterator(
    directory, data_generator, target_size=target_size,  color_mode='rgb', batch_size=batch_size, shuffle=True, seed=seed, 
    subset='validation'
)    

Found 23552 images belonging to 23 classes.
Found 5888 images belonging to 23 classes.


In [4]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(224,224,3)), 
    MaxPool2D(pool_size=(2, 2), strides=2), 
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'), 
    MaxPool2D(pool_size=(2,2), strides=2), 
    Flatten(), 
    Dense(units=23, activation='softmax'),
])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 200704)            0         
_________________________________________________________________
dense (Dense)                (None, 23)                4616215   
Total params: 4,635,607
Trainable params: 4,635,607
Non-trainable params: 0
______________________________________________

In [6]:
metrics=[tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=metrics)

In [7]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=20, steps_per_epoch=len(train_batches), verbose=1, workers=4)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 2356 steps, validate for 589 steps
Epoch 1/20
2242/2356 [===========================>..] - ETA: 5s - loss: 4.8268 - categorical_accuracy: 0.5335 - precision: 0.6409 - recall: 0.4450

C:\Users\Dennis\anaconda3\envs\ML\lib\site-packages\PIL\TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 19273 bytes but only got 816. Skipping tag 700
  " Skipping tag %s" % (size, len(data), tag)
C:\Users\Dennis\anaconda3\envs\ML\lib\site-packages\PIL\TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5140 bytes but only got 816. Skipping tag 34377
  " Skipping tag %s" % (size, len(data), tag)
C:\Users\Dennis\anaconda3\envs\ML\lib\site-packages\PIL\TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3144 bytes but only got 816. Skipping tag 34675
  " Skipping tag %s" % (size, len(data), tag)


2356/2356 [==============================] - 185s 79ms/step - loss: 4.6485 - categorical_accuracy: 0.5383 - precision: 0.6472 - recall: 0.4496 - val_loss: 1.0979 - val_categorical_accuracy: 0.6619 - val_precision: 0.7710 - val_recall: 0.5771
Epoch 2/20
2356/2356 [==============================] - 187s 79ms/step - loss: 0.8162 - categorical_accuracy: 0.7512 - precision: 0.8288 - recall: 0.6797 - val_loss: 0.8970 - val_categorical_accuracy: 0.7405 - val_precision: 0.8189 - val_recall: 0.6797
Epoch 3/20
2356/2356 [==============================] - 184s 78ms/step - loss: 0.5961 - categorical_accuracy: 0.8209 - precision: 0.8661 - recall: 0.7734 - val_loss: 0.9769 - val_categorical_accuracy: 0.7317 - val_precision: 0.7865 - val_recall: 0.6883
Epoch 4/20
2356/2356 [==============================] - 183s 78ms/step - loss: 0.4588 - categorical_accuracy: 0.8605 - precision: 0.8917 - recall: 0.8283 - val_loss: 0.9453 - val_categorical_accuracy: 0.7249 - val_precision: 0.8004 - val_recall: 0.6646

In [8]:
filepath = "./second_try_dennis.hdf5"

model.save(
   filepath, overwrite=True, include_optimizer=True, save_format="h5",
    signatures=None, options=None
)